# Get instruments pairs

In [20]:
import pandas as pd
import numpy as np
import requests
# Set the display option to show all columns
pd.set_option('display.max_columns', None)

In [21]:
def find_pairs(coin):
    url = "https://open-api.coinglass.com/public/v2/instrument"
    headers = {
        "accept": "application/json",
        "coinglassSecret": "084a0f80e01549299f538c575836abf8"
    }

    response = requests.get(url, headers=headers)
    data = response.json()

    pairs = []
    for exchange, instruments in data['data'].items():
        for item in instruments:
            if coin in [item['baseAsset'], item['quoteAsset']]:
                pairs.append({"exchange": exchange, "instrumentId": item['instrumentId']})

    return pd.DataFrame(pairs)

# Example usage:
df = find_pairs('BTC')
display(df)


,exchange,instrumentId
0,Binance,BTCUSD_231229
1,Binance,ETHBTC
2,Binance,BTCUSDT_240329
3,Binance,BTCUSD_240329
4,Binance,BTCBUSD
...,...,...
65,OKX,BTC-USDT-231110
66,OKX,BTC-USD-231110
67,OKX,BTC-USD-231117
68,OKX,BTC-USD-231229


## With the right pair, we can move to plots and dataframes

In [27]:
import time

current_time_millis = int(time.time() * 1000)
print(current_time_millis)


1699124972632


In [39]:
import requests
import pandas as pd

def fetch_data(exchange, pair, interval, limit=5, start_time=None, end_time=None):
    url = "https://open-api.coinglass.com/public/v2/indicator/open_interest_ohlc" 
    
    # Prepare query parameters
    params = {
        "ex": exchange,
        "pair": pair,
        "interval": interval,
        "limit": limit,
    }
    
    # Optionally add start and end time to the parameters
    if start_time is not None:
        params["start_time"] = start_time
    if end_time is not None:
        params["end_time"] = end_time
    
    # Set headers if needed
    headers = {
        "accept": "application/json",
    "coinglassSecret": "084a0f80e01549299f538c575836abf8"
}

    
    # Make the API request
    response = requests.get(url, params=params, headers=headers)
    
    # Handle the response
    if response.status_code == 200:
        response_json = response.json()
        
        # Assuming the data is present in a key named 'data'
        data = response_json.get('data', [])
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        # Convert timestamp to datetime
        if 't' in df.columns:
            df['t'] = pd.to_datetime(df['t'], unit='ms')
        return df
    else:
        response.raise_for_status()

# Example usage:
df = fetch_data("Binance", "BTCUSDT", "h24", limit=100, start_time="1668481704000", end_time="1699124972632")
print(df.head())


           t          o          c          h          l
0 2022-11-16  117744.17  123207.73  126189.60  117715.55
1 2022-11-17  123207.73  124497.30  126468.56  123046.99
2 2022-11-18  124497.30  120831.50  125897.51  120822.02
3 2022-11-19  120831.50  119623.04  121515.97  119289.84
4 2022-11-20  119623.04  121296.05  121315.33  118522.40


In [40]:
url = "https://open-api.coinglass.com/public/v2/indicator/price_ohlc?ex=Binance&pair=BTCUSDT&interval=h24&limit=500&start_time=158767867876&end_time=1668568104000"

headers = {
    "accept": "application/json",
    "coinglassSecret": "084a0f80e01549299f538c575836abf8"
}

response = requests.get(url, headers=headers)

print(response.text)

{"code":"0","msg":"success","data":[[1569369600,"8493.32","8725.24","8200","8430.07","406841550.16218"],[1569456000,"8428.15","8464.4","7700.67","8060.15","487919029.55307"],[1569542400,"8061.98","8260","7851.94","8176.52","321440849.1052"],[1569628800,"8178.26","8311","8001.14","8195.38","275406180.40421"],[1569715200,"8199","8226.35","7888.98","8041.96","289320913.71819"],[1569801600,"8042.08","8331.89","7709.01","8285.84","436701925.16117"],[1569888000,"8285.31","8499","8171","8290","327218811.62991"],[1569974400,"8290","8365.92","8149.35","8352.14","249785989.08192"],[1570060800,"8353.4","8386","8020","8220.27","274334928.39047"],[1570147200,"8219.95","8227.42","7979.95","8130.56","230978393.31482"],[1570233600,"8130","8175","8010","8122.14","244744089.06028"],[1570320000,"8122.01","8148.43","7760","7852.79","333155725.01068"],[1570406400,"7851.01","8330","7764.42","8186.88","504822438.82757"],[1570492800,"8188.55","8322.01","8103.88","8167.62","366576763.24705"],[1570579200,"8168.

In [46]:
df = pd.DataFrame(response.json()["data"])
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], unit="ms")
df.head(20)

,0,1,2,3,4,5
0,1970-01-19 03:56:09.600,8493.32,8725.24,8200,8430.07,406841550.16218
1,1970-01-19 03:57:36.000,8428.15,8464.4,7700.67,8060.15,487919029.55307
2,1970-01-19 03:59:02.400,8061.98,8260,7851.94,8176.52,321440849.1052
3,1970-01-19 04:00:28.800,8178.26,8311,8001.14,8195.38,275406180.40421
4,1970-01-19 04:01:55.200,8199,8226.35,7888.98,8041.96,289320913.71819
5,1970-01-19 04:03:21.600,8042.08,8331.89,7709.01,8285.84,436701925.16117
6,1970-01-19 04:04:48.000,8285.31,8499,8171,8290,327218811.62991
7,1970-01-19 04:06:14.400,8290,8365.92,8149.35,8352.14,249785989.08192
8,1970-01-19 04:07:40.800,8353.4,8386,8020,8220.27,274334928.39047
9,1970-01-19 04:09:07.200,8219.95,8227.42,7979.95,8130.56,230978393.31482
